In [1]:
import pandas as pd
import numpy as np
import random as rd
import geopandas as gpd
import requests

In [2]:
df = pd.read_csv("downloads/house-details.csv")
lf = pd.read_csv("generated/lat-lon.csv")

/tmp/ipykernel_90196/3197607116.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("downloads/house-details.csv")


In [3]:
df = pd.concat([df, lf], axis = 1)

In [4]:
df = df[["ST_NUM", "ST_NAME", "CITY", "ROOF_STRUCTURE", "TOTAL_VALUE", "lat", "long"]].dropna()

In [5]:
df.loc[df["ROOF_STRUCTURE"] == "F - Flat", 'capacity'] = 1529
df.loc[df["ROOF_STRUCTURE"] == "H - Hip", 'capacity'] = 277
df.loc[df["ROOF_STRUCTURE"] == "M - Mansard", 'capacity'] = 16
df.loc[df["ROOF_STRUCTURE"] == "G - Gable", 'capacity'] = 173
df.loc[df["ROOF_STRUCTURE"] == "L - Gambrel", 'capacity'] = 24
df.loc[df["ROOF_STRUCTURE"] == "S - Shed", 'capacity'] = 119
df.loc[df["ROOF_STRUCTURE"] == "O - Other", 'capacity'] = 42

df

,ST_NUM,ST_NAME,CITY,ROOF_STRUCTURE,TOTAL_VALUE,lat,long,capacity
0,104.0,PUTNAM ST,EAST BOSTON,F - Flat,"792,000",42.379589,-71.031984,1529.0
1,197.0,Lexington ST,EAST BOSTON,F - Flat,"818,200",42.379519,-71.032154,1529.0
2,199.0,Lexington ST,EAST BOSTON,F - Flat,"804,400",42.379561,-71.032304,1529.0
3,201.0,Lexington ST,EAST BOSTON,M - Mansard,"735,300",42.379452,-71.032552,16.0
4,203.0,Lexington ST,EAST BOSTON,M - Mansard,"731,600",42.379686,-71.031737,16.0
...,...,...,...,...,...,...,...,...
17438,72.0,Washington ST,CHARLESTOWN,F - Flat,"2,057,500",42.360638,-71.063084,1529.0
17439,56.0,Union ST,CHARLESTOWN,F - Flat,0,42.360789,-71.063071,1529.0
17440,56.0,Union ST,CHARLESTOWN,F - Flat,"1,089,400",42.360789,-71.063071,1529.0
17441,56.0,Union ST,CHARLESTOWN,F - Flat,"970,700",42.360638,-71.063084,1529.0


In [6]:
df["street_num"] = df["ST_NUM"]
df["street_name"] = df["ST_NAME"]
df["city"] = df["CITY"]
df = df.drop(["ST_NUM", "ST_NAME", "CITY", "ROOF_STRUCTURE"], axis = 1)

In [7]:
df["TOTAL_VALUE"] = df["TOTAL_VALUE"].str.replace(',', '').astype(int)
df["TOTAL_VALUE"] = df["TOTAL_VALUE"] // 1000

buckets = [k * 1000 for k in range(0, 50)]
labels = [k * 1000 for k in range(1, 50)]
df["income_bucket"] = pd.cut(df["TOTAL_VALUE"], bins = buckets, labels = labels)

df

,TOTAL_VALUE,lat,long,capacity,street_num,street_name,city,income_bucket
0,792,42.379589,-71.031984,1529.0,104.0,PUTNAM ST,EAST BOSTON,1000
1,818,42.379519,-71.032154,1529.0,197.0,Lexington ST,EAST BOSTON,1000
2,804,42.379561,-71.032304,1529.0,199.0,Lexington ST,EAST BOSTON,1000
3,735,42.379452,-71.032552,16.0,201.0,Lexington ST,EAST BOSTON,1000
4,731,42.379686,-71.031737,16.0,203.0,Lexington ST,EAST BOSTON,1000
...,...,...,...,...,...,...,...,...
17438,2057,42.360638,-71.063084,1529.0,72.0,Washington ST,CHARLESTOWN,3000
17439,0,42.360789,-71.063071,1529.0,56.0,Union ST,CHARLESTOWN,NaN
17440,1089,42.360789,-71.063071,1529.0,56.0,Union ST,CHARLESTOWN,2000
17441,970,42.360638,-71.063084,1529.0,56.0,Union ST,CHARLESTOWN,1000


In [8]:
df = df.drop("TOTAL_VALUE", axis = 1)
df = df.dropna()

In [9]:
bpps = df.sample(frac = 0.3)
baps = df.drop(bpps.index)

In [10]:
bpps[["lat", "long"]].to_csv("simulator/bpp/position.csv", index=False)
bpps["capacity"].astype(float).to_csv("simulator/bpp/capacity.csv", index=False)

In [11]:
baps[["lat", "long"]].to_csv("simulator/bap/position.csv", index=False)
baps["income_bucket"].astype(float).to_csv("simulator/bap/income.csv", index=False)

In [12]:
# df2 = pd.read_csv("utility-bills.csv")

# df2[df2["EnergyTypeName"] == "Electric"]["TotalCost"].describe()

In [ ]:
# get a geo plot of all the streets in boston
gdf = gpd.read_file('roofs.geojson')
gdf.plot(figsize=(16, 16), color="gray")